In [1]:
from calendar import month
from operator import index
import snscrape.modules.twitter as sntwitter
import pandas as pd
import glob
import datetime

release_dates = pd.read_csv("Release dates - data.csv")
release_dates_map = {}

folderpathrt = "rotten_info/*"
filesrt = []

for file in glob.glob(folderpathrt):
    filesrt.append(file)

for file in filesrt:
    print(file)
    movies = pd.read_csv(file)

    tweets = []
    for ind, movie in movies.iterrows():
        movien = movie['Release Group']
        if not pd.isna(movie['Release Date (Theaters)']):
            release_date = ('').join(movie['Release Date (Theaters)'].strip('[]').split(',')).replace("'", "")
            rd = datetime.datetime.strptime(release_date, "%b %d %Y")
            # if release_dates_map.get(movien.lower(), ''):
            release_dates_map[movien.lower()] = rd

folderpath = "Box Office Mojo/*"
files = []

for file in glob.glob(folderpath):
    files.append(file)

for ind, row in release_dates.iterrows():
    if row['Release-Date']:
        rd = datetime.datetime.strptime(row['Release-Date'], "%Y-%m-%d")
        if not release_dates_map.get(row['Movie'].lower(), None):
            release_dates_map[row['Movie'].lower()] = rd
    
count = 0
c = 0
for file in files:
    if '2019' in file:
        print(file)
        movies = pd.read_csv(file)

        tweets = []
        for ind, movie in movies.iterrows():
            c += 1
            query_txt = movie['Release Group']
            release_date = release_dates_map.get(movie['Release Group'].lower(), None)
            if release_date:
                count += 1
                # rd = datetime.datetime.strptime(release_date, "%Y-%m-%d")
                since = release_date - datetime.timedelta(weeks=4)
                until = release_date + datetime.timedelta(weeks=1)
                query_txt += " since:%s until:%s" % (since.date(), until.date())
                print(movie['Release Group'], release_date.date())

                for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query_txt).get_items()):
                    if i>5000:
                        break
                    tweets.append([movie['Release Group'], tweet.date, tweet.id, tweet.content, tweet.user.username])

        tweets_df = pd.DataFrame(tweets, columns=['Movie Name', 'Datetime', 'Tweet Id', 'Text', 'Username'])
        tweets_df.to_csv("Twitter/tweets_%s" % file.split('/')[1], index=False, escapechar=r'|')
        break


Box Office Mojo/2019.csv
Avengers: Endgame 2019-04-26
The Lion King 2019-07-19
Frozen II 2019-11-22
Spider-Man: Far from Home 2019-07-02
Captain Marvel 2019-03-08
Joker 2019-10-04
Star Wars: Episode IX - The Rise of Skywalker 2019-12-20
Toy Story 4 2019-06-21
Aladdin 2019-05-24
Jumanji: The Next Level 2019-12-13
Fast & Furious Presents: Hobbs & Shaw 2019-08-02
Ne Zha 2019-08-29
The Wandering Earth 2019-02-05
How to Train Your Dragon: The Hidden World 2019-02-22
Maleficent: Mistress of Evil 2019-10-18
It Chapter Two 2019-09-06
Pokémon Detective Pikachu 2019-05-10
The Secret Life of Pets 2 2019-06-07
Alita: Battle Angel 2019-02-14
Godzilla: King of the Monsters 2019-05-31
1917 2020-12-25
Once Upon a Time... In Hollywood 2019-07-26
Shazam! 2019-04-05
Dumbo 2019-03-29
John Wick: Chapter 3 - Parabellum 2019-05-17
Knives Out 2019-11-27
Terminator: Dark Fate 2019-11-01
Parasite 2019-10-11
Us 2019-03-22
Men in Black: International 2019-06-14
X-Men: Dark Phoenix 2019-06-07
Glass 2019-01-18
Anna